In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# GPU를 사용할 수 있으면 사용하고, 없으면 CPU 사용
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 이미지 데이터를 0~1로 정규화하고 텐서로 변환
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# FashionMNIST 데이터셋 불러오기 (훈련용, 테스트용 나눠서)
train_data = datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
test_data = datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)

# 데이터 배치로 나눠서 로딩 (한 번에 128개씩 처리)
train_loader = DataLoader(train_data, batch_size=128, shuffle=True)
test_loader = DataLoader(test_data, batch_size=128, shuffle=False)

# CNN(합성곱 신경망) 모델 정의
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        # 1채널 입력 이미지를 32채널로 변환 (3x3 필터, stride=1)
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        # 32채널 이미지를 64채널로 변환
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        # 과적합 방지를 위해 일부 노드를 랜덤으로 꺼줌
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        # fully connected layer: conv에서 나온 결과를 일렬로 펼쳐 넣음
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)  # 10개의 옷 종류(클래스) 분류

    def forward(self, x):
        x = self.conv1(x)          # 첫 번째 합성곱
        x = nn.functional.relu(x) # 비선형 활성화(ReLU)
        x = self.conv2(x)          # 두 번째 합성곱
        x = nn.functional.relu(x)
        x = nn.functional.max_pool2d(x, 2)  # 2x2 최대풀링 (특징 압축)
        x = self.dropout1(x)      # Dropout 적용
        x = torch.flatten(x, 1)   # 1차원으로 펼치기
        x = self.fc1(x)           # 첫 번째 완전 연결층
        x = nn.functional.relu(x)
        x = self.dropout2(x)      # Dropout 적용
        x = self.fc2(x)           # 두 번째 완전 연결층 (최종 출력)
        output = nn.functional.log_softmax(x, dim=1)  # softmax로 확률 변환
        return output

# 모델, 손실 함수, 최적화 알고리즘 설정
model = CNNModel().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.0005)
loss_fn = nn.CrossEntropyLoss()

# 학습할 epoch 수
epochs = 20

for epoch in range(epochs):
    model.train()  # 학습 모드 켜기
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()           # 이전 gradient 초기화
        output = model(data)            # forward (예측값 계산)
        loss = loss_fn(output, target)  # loss 계산
        loss.backward()                 # gradient 계산
        optimizer.step()                # 파라미터 업데이트

    # 한 epoch이 끝나면 테스트 데이터로 성능 확인
    model.eval()  # 평가 모드 켜기
    test_loss = 0
    correct = 0
    with torch.no_grad():  # gradient 계산 끄기 (메모리 절약, 속도 ↑)
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += loss_fn(output, target).item()  # batch별 loss 누적
            pred = output.argmax(dim=1, keepdim=True)   # 예측 클래스 선택
            correct += pred.eq(target.view_as(pred)).sum().item()  # 맞춘 개수 카운트

    test_loss /= len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)
    print(f'Epoch {epoch+1}: Test Loss: {test_loss:.4f}, Accuracy: {accuracy:.2f}%')


100%|██████████| 26.4M/26.4M [00:01<00:00, 21.5MB/s]
100%|██████████| 29.5k/29.5k [00:00<00:00, 336kB/s]
100%|██████████| 4.42M/4.42M [00:00<00:00, 6.12MB/s]
100%|██████████| 5.15k/5.15k [00:00<00:00, 17.9MB/s]


Epoch 1: Test Loss: 0.0028, Accuracy: 86.96%
Epoch 2: Test Loss: 0.0024, Accuracy: 89.01%
Epoch 3: Test Loss: 0.0022, Accuracy: 89.98%
Epoch 4: Test Loss: 0.0021, Accuracy: 90.40%
Epoch 5: Test Loss: 0.0020, Accuracy: 90.48%
Epoch 6: Test Loss: 0.0019, Accuracy: 91.27%
Epoch 7: Test Loss: 0.0019, Accuracy: 91.41%
Epoch 8: Test Loss: 0.0018, Accuracy: 91.72%
Epoch 9: Test Loss: 0.0017, Accuracy: 92.04%
Epoch 10: Test Loss: 0.0017, Accuracy: 92.18%
Epoch 11: Test Loss: 0.0018, Accuracy: 92.14%
Epoch 12: Test Loss: 0.0017, Accuracy: 92.31%
Epoch 13: Test Loss: 0.0017, Accuracy: 92.04%
Epoch 14: Test Loss: 0.0017, Accuracy: 92.55%
Epoch 15: Test Loss: 0.0017, Accuracy: 92.67%
Epoch 16: Test Loss: 0.0017, Accuracy: 92.74%
Epoch 17: Test Loss: 0.0017, Accuracy: 92.73%
Epoch 18: Test Loss: 0.0017, Accuracy: 92.67%
Epoch 19: Test Loss: 0.0018, Accuracy: 92.69%
Epoch 20: Test Loss: 0.0017, Accuracy: 92.99%
